In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime

f = open("iris.data", "r")  
data = f.read()             
data_list = data.split("\n")

seperated = []
virginica = []
versicolor = []
setosa = []

for i in range(len(data_list)):
    seperated = data_list[i].split(",")
    sl = float(seperated[0])
    sw = float(seperated[1])
    pl = float(seperated[2])
    pw = float(seperated[3])
    #sınıfları belirtelim
    if seperated[4] == "Iris-virginica":
        virginica.append([sl, sw, pl, pw, 0])
    elif seperated[4] == "Iris-versicolor":
        versicolor.append([sl, sw, pl, pw, 1])
    elif seperated[4] == "Iris-setosa":
        setosa.append([sl, sw, pl, pw, 2])

data = np.concatenate((versicolor, np.concatenate((virginica, setosa))))    

df = pd.DataFrame(data, columns=['SepalLength', 'SepalWidth', 'PetalLength', "PetalWidth", "Name"]) 

X = df.drop('Name', axis=1).values
y = df['Name'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  

X_train = torch.FloatTensor(X_train)    
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=4, out_features=5) 
        self.fc2 = nn.Linear(in_features=5, out_features=4)
        self.output = nn.Linear(in_features=4, out_features=3)

    #aktivasyon fonksiyonu
    def forward(self, x):
        x = F.relu(self.fc1(x)) #ilk durumda relu yerine sigmoid kullanmıştık
        x = F.relu(self.fc2(x)) #relu yerine sigmoid kullanmıştık
        x = self.output(x)
        return x

model = ANN()

criterion = nn.CrossEntropyLoss()   #hata fonksiyonu

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

epochs = 1000
loss_arr = []
for i in range(epochs):
    y_pred = model.forward(X_train) #ileri yol
    loss = criterion(y_pred, y_train)
    loss_arr.append(loss)

    if loss < 0.0001:   #durdurma kriteri
        break

    optimizer.zero_grad()
    loss.backward()     #geri yol
    optimizer.step()
print(f'Loss: {loss}')

predict_out = model(X_test) #model testi
_, predict_y = torch.max(predict_out, 1)

print('prediction accuracy', accuracy_score(y_test.data, predict_y.data))



Loss: 0.04508201405405998
prediction accuracy 1.0
